# ExxonMobil Permian Production in Texas and New Mexico

In [159]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [160]:
# get the cleaned data (see clean_data.py; detail on data source and assumptions in README.md)
tx_df = pd.read_csv(
    "/Users/lindseygulden/dev/leg-up-private/texas_rrc_district_level_oil_and_gas_production_by_month_and_operator.csv",
)
nm_df = pd.read_csv(
    "/Users/lindseygulden/dev/leg-up-private/nm_ocd_statewide_oil_and_gas_production_by_month_and_operator.csv"
)

# Check our methods by doing an apples-to-apples comparison between our assembled queried data and ExxonMobil's public statements.
### Compute 'Permian total' MBOE/day rate for the first 9 months of 2023 (which is what is referenced by the XOM press release on Oct 11, 2023)
[ExxonMobil Announces Merger with Pioneer Natural Resources: press release 10/11/2023](https://investor.exxonmobil.com/news-events/press-releases/detail/1147/exxonmobil-announces-merger-with-pioneer-natural-resources)

"At close, ExxonMobil’s Permian production volume would more than double to 1.3 million barrels of oil equivalent per day (MOEBD), based on 2023 volumes, and is expected to increase to approximately 2 MOEBD in 2027."

In [161]:
# Find production totals, in units of MBOE per day, for each operator in Texas and New Mexico for first 9 months of 2023 (ahead of XOM press release)
nm_tmp_df = (
    nm_df[["state", "boe", "operator", "n_days"]]
    .loc[(nm_df.year == 2023) & (nm_df.month < 10)]
    .groupby(["state", "operator"])
    .sum()
).reset_index()
tx_tmp_df = (
    tx_df[["state", "boe", "operator", "district", "n_days"]]
    .loc[(tx_df.year == 2023) & (tx_df.month < 10)]
    .groupby(["state", "operator", "district"])
    .sum()
    .reset_index()
    .groupby(["state", "operator"])
    .agg({"boe": "sum", "n_days": "mean"})
).reset_index()

# concatenate tx and nm
both_states_df = pd.concat([nm_tmp_df, tx_tmp_df])

# sum total production (units BOE/day) for permian across XTO and Pioneer for first nine months of year
production_before_10_2023_df = both_states_df.loc[
    (both_states_df.state == "TX")
    | (both_states_df.operator == "XTO PERMIAN OPERATING LLC.")
].agg({"boe": "sum", "n_days": "mean"})

# compute BOE/day rate
mboe_rate = (
    production_before_10_2023_df["boe"] / production_before_10_2023_df["n_days"]
) / 1e6

print(
    f"For the first 9 months of 2023, ahead of XOM press release on Oct 13, 2023, the rate of Permian production was: {mboe_rate:.1f} Mboe/day."
)

For the first 9 months of 2023, ahead of XOM press release on Oct 13, 2023, the rate of Permian production was: 1.3 Mboe/day.


In [163]:
# Now compute 2024 production of the non-Pioneer Permian assets, in units of bbls of oil produced per day. (That is, neglect BOE conversions of gas.)

# align column names for produced oil
nm_df.rename(columns={"oil_produced_bbl": "oil_bbl"}, inplace=True)

nm_tmp_df = (
    nm_df[["state", "oil_bbl", "operator", "n_days"]]
    .loc[(nm_df.year == 2024)]
    .groupby(["state", "operator"])
    .sum()
).reset_index()
tx_tmp_df = (
    tx_df[["state", "oil_bbl", "operator", "district", "n_days"]]
    .loc[(tx_df.year == 2024)]
    .groupby(["state", "operator", "district"])
    .sum()
    .reset_index()
    .groupby(["state", "operator"])
    .agg({"oil_bbl": "sum", "n_days": "mean"})
).reset_index()

# concatenate tx and nm
both_states_df = pd.concat([nm_tmp_df, tx_tmp_df])

# sum 2024 production (units bbl oil/day) for permian across XTO ENERGY in Texas and XTO PERMIAN OPERATING LLC in NM (i.e., the historical Permian acreage)
production_2024_df = both_states_df.loc[
    ((both_states_df.state == "TX") & (both_states_df.operator == "XTO ENERGY INC."))
    | (both_states_df.operator == "XTO PERMIAN OPERATING LLC.")
].agg({"oil_bbl": "sum", "n_days": "mean"})

# compute bbl oil/day rate for 2024
bbl_per_day_rate = production_2024_df["oil_bbl"] / production_2024_df["n_days"]

print(
    f"In 2024 (Jan-Nov), the rate of Permian oil production from ExxonMobil's non-Pioneer Permian assets was: {bbl_per_day_rate:.0f} bbl/day."
)

In 2024 (Jan-Nov), the rate of Permian oil production from ExxonMobil's non-Pioneer Permian assets was: 274092 bbl/day.
